In [1]:
import pandas as pd

data_theatre = pd.read_csv('DataTheatre_dates_prix.csv')

adresses_uniques = data_theatre['adresse'].unique()

In [2]:
adresses_uniques = data_theatre['adresse'].unique()

In [4]:
import requests

def obtenir_coordonnees_adresses(adresses):
    coordonnees = {}

    for adresse in adresses:
        if adresse not in coordonnees:
            base_url = "https://api-adresse.data.gouv.fr/search/"
            params = {
                "q": adresse + ', Paris, France',
                "limit": 1
            }

            response = requests.get(base_url, params=params)

            if response.status_code == 200:
                data = response.json()
                if data.get('features'):
                    coordinates = data['features'][0]['geometry']['coordinates']
                     
                    coordonnees[adresse] = tuple(reversed(coordinates))
                else:
                    coordonnees[adresse] = None
            else:
                print(f"Erreur lors de la requête pour l'adresse {adresse} : {response.status_code}")
                coordonnees[adresse] = None
                
    return coordonnees


coordonnees = obtenir_coordonnees_adresses(adresses_uniques)

data_theatre['Coordonnees'] = data_theatre['adresse'].map(coordonnees)

data_theatre.to_csv('DataTheatre_base_finale.csv', sep=';', index=False)